In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import math
import wave
from scipy.signal import find_peaks
from sklearn.neighbors import KernelDensity
import librosa

In [4]:
offset = 0/ 44100

location = "crewcamp/samsstag" 
gyro = pd.read_csv(location + "/android.sensor.gyroscope.csv")
accel = pd.read_csv(location + "/android.sensor.accelerometer.csv")
beats = pd.read_csv(location + "/out.csv")
accel_time = accel.iloc[:,0] / 1000000000  - offset
gyro_time = gyro.iloc[:,0] / 1000000000 - offset
offset / 60


0.0

3_114_945
19_217_450
36_950_701
53_911_592
73_658_030
91_545_764
107_023_659
124_238_744
140_475_164
154_035_963

In [3]:
y, sr = librosa.load(location + "/cut.wav")
tempo, beats_ = librosa.beat.beat_track(y=y, sr=sr)
# beats now contains the beat *frame positions*
# convert to timestamps like this:
beat_times = librosa.frames_to_time(beats_, sr=sr)

In [154]:
beat_diff = (beats.iloc[-1,0] - beats.iloc[0,0]) / (len(beats.iloc[:,0])-1)
beats = beats + beat_diff*0.5

In [4]:
beat_diff = (beat_times[-1] - beat_times[0]) / (len(beat_times)-1)
beat_times = beat_times + beat_diff*0.5

In [5]:
min_prominence = None
min_height = None

In [28]:
#min_height = -5

In [6]:
gyro_length = np.sqrt(gyro.iloc[:,1] **2 + gyro.iloc[:,2] **2 + gyro.iloc[:,3] **2)

gyro_length_valleys_all, temp = find_peaks(-1 * gyro_length,height=(None,None),prominence=(None,None))
gyro_length_valleys_time_all = gyro_time[gyro_length_valleys_all]
gyro_length_valleys_height_all = temp["peak_heights"]
gyro_length_valleys_prominence_all = temp["prominences"]

accel_length = np.sqrt(accel.iloc[:,1] **2 + accel.iloc[:,2] **2 + accel.iloc[:,3] **2)

In [7]:
kde = KernelDensity(kernel='epanechnikov',bandwidth='silverman').fit(np.array(gyro_length_valleys_prominence_all).reshape((-1,1)))
x_plot = np.linspace(math.floor(min(gyro_length_valleys_prominence_all)),math.ceil(max(gyro_length_valleys_prominence_all)),1000)[:,np.newaxis]
log_dens = kde.score_samples(x_plot)

peaks , _ = find_peaks(-1 * np.exp(log_dens),height=-0.02)
min_prominence= x_plot[peaks[0]][0]

plt.close()
plt.plot(x_plot[:,0],np.exp(log_dens))
plt.plot(x_plot[peaks[0]],np.exp(log_dens)[peaks[0]],"rx",label="first valley under 0.02")
#plt.plot(gyro_length_valleys_prominence_all, -0.005 - 0.05 * np.random.random(gyro_length_valleys_prominence_all.shape[0]), "+k")
plt.xlabel("prominence")
plt.ylabel("density")
plt.legend()
#plt.savefig("prominence_graph.pdf", format="pdf")
plt.show()


In [8]:
gyro_length_valleys_filer, temp = find_peaks(-1 * gyro_length,height=(None,None),prominence=(min_prominence,None))
gyro_length_valleys_time_filer = gyro_time[gyro_length_valleys_filer]
gyro_length_valleys_height_filer = temp["peak_heights"]
gyro_length_valleys_prominence_filer = temp["prominences"]

In [9]:
temp_gyro_length_valleys_height = gyro_length_valleys_height_filer * (-1)
kde = KernelDensity(kernel='epanechnikov',bandwidth='silverman').fit(np.array(temp_gyro_length_valleys_height).reshape((-1,1)))
x_plot = np.linspace(math.floor(min(temp_gyro_length_valleys_height)),math.ceil(max(temp_gyro_length_valleys_height)),1000)[:,np.newaxis]
log_dens = kde.score_samples(x_plot)

peaks , temp = find_peaks(np.exp(log_dens),height=(None,None))
height = temp["peak_heights"]
leargest_height = peaks[np.where(height == max(height))[0][0]]

peaks , temp = find_peaks(-1 * np.exp(log_dens[leargest_height:]),prominence=(None,None))
#min_prominence= x_plot[peaks[0]][0]
prominences = temp["prominences"]

leargest_prominence = np.where(prominences == max(prominences))

min_height = -1 * (x_plot[peaks[leargest_prominence] + leargest_height][0][0])

plt.close()
plt.plot(x_plot[:,0],np.exp(log_dens))
plt.plot(x_plot[peaks[leargest_prominence] + leargest_height],np.exp(log_dens)[peaks[leargest_prominence] + leargest_height],"rx",label="valley with biggest prominence")
#plt.plot(temp_gyro_length_valleys_height, -0.005 - 0.05 * np.random.random(temp_gyro_length_valleys_height.shape[0]), "+k")
plt.vlines(x_plot[peaks + leargest_height],np.exp(log_dens)[peaks + leargest_height],np.exp(log_dens)[peaks + leargest_height] + prominences,"g",label="prominences of all valleys")
plt.xlabel("height")
plt.ylabel("density")
plt.legend()
#plt.savefig("height_kde.pdf", format="pdf")
plt.show()



In [10]:
gyro_length_valleys, temp = find_peaks(-1 * gyro_length,height=(min_height,None),prominence=(min_prominence,None))
gyro_length_valleys_time = gyro_time[gyro_length_valleys]
gyro_length_valleys_height = temp["peak_heights"]
gyro_length_valleys_prominence = temp["prominences"]

In [11]:
def find_most_prominent_peak(peaks,peaks_prominences,valleys = None,valleys_prominences = None,min_threshold = 0,max_treshold = math.inf):
    max_prominence = -math.inf
    # 
    max_peak = None
    
    offset = (min_threshold + max_treshold) / 2
    for peak, prominence in zip(peaks, peaks_prominences):
        if peak > max_treshold:
            continue
        if peak > min_threshold and prominence * calc_prominence_factor(peak,offset=offset) > max_prominence:
            max_prominence = prominence * calc_prominence_factor(peak,offset=offset)
            max_peak = peak

    if valleys != None:
        for peak, prominence in zip(valleys, valleys_prominences):
            if peak > max_treshold:
                continue
            if peak > min_threshold and prominence * calc_prominence_factor(peak,offset=offset) > max_prominence:
                max_prominence = prominence * calc_prominence_factor(peak,offset=offset)
                max_peak = peak

    return (max_peak, max_prominence)
    
def calc_prominence_factor(x,threshold = 0.2,offset=0):
    return  math.exp(0-100*(x-offset)**2)

In [12]:
bps = []

for i in range(len(beat_times)):
    count = 2 
    if i > 0:
        prev = beat_times[i-1]
    else:
        prev =  beat_times[i]
        count = 1
    if i+1 < len(beat_times):
        next = beat_times[i+1]
    else:
        next = beat_times[i]
        count = 1
    diff = (next - prev) /count
    bps.append(1/diff)

In [16]:
bps = []

for i in range(len(beats.iloc[:,0])):
    count = 2 
    if i > 0:
        prev = beats.iloc[i-1,0]
    else:
        prev =  beats.iloc[i,0]
        count = 1
    if i+1 < len(beats.iloc[:,0]):
        next = beats.iloc[i+1,0]
    else:
        next = beats.iloc[i,0]
        count = 1
    diff = (next - prev) /count
    bps.append(1/diff)

In [13]:
min_threshold = 0
max_threshold = math.inf

beat_diff = (beats.iloc[-1,0] - beats.iloc[0,0]) / (len(beats.iloc[:,0])-1)
threshold = beat_diff / 2

gyro_peaks_with_prominence = []

for beat,tempo in zip(beat_times,bps):
#for beat,tempo in zip(beats.iloc[:,0],bps):

    min_threshold = beat - 0.5/tempo
    max_threshold = beat + 0.5/tempo

    gyro_peaks_with_prominence.append((find_most_prominent_peak(gyro_length_valleys_time,gyro_length_valleys_height,min_threshold=min_threshold,max_treshold=max_threshold),beat,tempo))



In [14]:
bandwith = 10
averages = []
beats_avarage = []
elements = []
abs_running_average = []
all_used_peaks = []
temp = []
for (peak, _), beat, tempo in gyro_peaks_with_prominence:
    if peak == None:
        elements = []
        if len(averages)>0:
            abs_running_average.append((averages,beats_avarage))
            averages = []
            beats_avarage = []
            all_used_peaks.extend(temp)
        temp = []
        continue
    prominence_factor =1#  (min(prominence / (self.global_max_prominence * 0.5) , 1)) ** 100
    elements.append((abs(peak - beat)*tempo,prominence_factor))
    temp.append((abs(peak - beat)*tempo))
    if len(elements) > bandwith:
        elements = elements[1:]
    if len(elements) > 2:
        sum_prominence = 0
        count = 0
        for i, j in elements:                
            sum_prominence += i 
            count += 1
        average = sum_prominence / count
        averages.append(average)
        beats_avarage.append(beat)
if len(averages)>0:
    abs_running_average.append((averages,beats_avarage))
    all_used_peaks.extend(temp)
abs_global_average = sum(all_used_peaks) / len(all_used_peaks)

In [15]:
fig, ax = plt.subplots()


#plt.axhline(threshold)
plt.xlabel("time in seconds")
plt.ylabel("distance to next beat in beats")
plt.title("time delta for each beat")
plt.ylim([0, 0.5])
#plt.axhline(-1*beat_diff)
#plt.axhline(-1*threshold)

props = dict(boxstyle='round', facecolor='wheat', alpha=0.75)

max_bpm = max(bps)*60
min_bpm = min(bps)*60
average_bpm = 60 *  sum(bps)/len(bps)

textstr = '\n'.join((r'%.4f ms' % ((abs_global_average  * 1000)*beat_diff,),
                     r'%.4f beats' % (abs_global_average,),
                     r'%.2f max bpm' % (max_bpm,),
                     r'%.2f min bpm' % (min_bpm,),
                     r'%.2f avg bpm' % (average_bpm,)))

plt.text(0.05, 0.75,textstr, transform=ax.transAxes,bbox=props)

label = True
for (peak, _),beat,tempo in gyro_peaks_with_prominence:
    if (peak == None):
        continue
    diff = abs(peak - beat)
    if label:
        plt.plot(beat, diff * tempo,"bx", label="beat")
        label = False
    else:
        plt.plot(beat, diff * tempo,"bx") 


for averages, beats_ in abs_running_average:

     plt.plot(beats_,averages,"r")

plt.legend()
plt.savefig("result.pdf",format="pdf")
plt.show()

: 

In [117]:
bandwith = 10
averages = []
beats_avarage = []
elements = []
running_average = []
all_used_peaks = []
temp = []
for (peak, _), beat,tempo in gyro_peaks_with_prominence:
    if peak == None:
        elements = []
        if len(averages)>0:
            running_average.append((averages,beats_avarage))
            averages = []
            beats_avarage = []
            all_used_peaks.extend(temp)
        temp = []
        continue
    prominence_factor =1#  (min(prominence / (self.global_max_prominence * 0.5) , 1)) ** 100
    elements.append(((peak - beat)*tempo,prominence_factor))
    temp.append(((peak - beat)*tempo))
    if len(elements) > bandwith:
        elements = elements[1:]
    if len(elements) > 2:
        sum_prominence = 0
        count = 0
        for i, j in elements:                
            sum_prominence += i 
            count += 1
        average = sum_prominence / count
        averages.append(average)
        beats_avarage.append(beat)
if len(averages)>0:
    running_average.append((averages,beats_avarage))
    all_used_peaks.extend(temp)
global_average = sum(all_used_peaks) / len(all_used_peaks)

In [118]:
fig, ax = plt.subplots()

plt.axhline(0,color="k")

#plt.axhline(threshold)
plt.xlabel("time in seconds")
plt.ylabel("distance to next beat in beats")
plt.title("time delta for each beat")
plt.ylim([-0.5, 0.5])
#plt.axhline(-1*beat_diff)
#plt.axhline(-1*threshold)

props = dict(boxstyle='round', facecolor='wheat', alpha=0.75)

max_bpm = max(bps)*60
min_bpm = min(bps)*60
average_bpm = 60 *  sum(bps)/len(bps)

textstr = '\n'.join((r'%.4f ms' % ((global_average  * 1000)*beat_diff,),
                     r'%.4f beats' % (global_average,),
                     r'%.2f max bpm' % (max_bpm,),
                     r'%.2f min bpm' % (min_bpm,),
                     r'%.2f avg bpm' % (average_bpm,)))

plt.text(0.05, 0.75,textstr, transform=ax.transAxes,bbox=props)

label = True
for (peak, _),beat,tempo in gyro_peaks_with_prominence:
    if (peak == None):
        continue
    diff = (peak - beat)*tempo
    if label:
        plt.plot(beat, diff,"bx", label="beat")
        label = False
    else:
        plt.plot(beat, diff,"bx") 


for averages, beats_ in running_average:

     plt.plot(beats_,averages,"r")


plt.legend()
plt.show()

In [87]:
fig, axs = plt.subplots(ncols = 1, nrows = 4,sharey=True,sharex=True)

fig.suptitle('gyro with peaks with prominence > 25', fontsize=16)

fig.supxlabel("time in seconds")
fig.supylabel("gyro")

for x, y in zip(beats.iloc[:,0], beats.iloc[:,1]):

    seconds = x#+ offset_seconds

    if (y == 1):
        linestyle = 'solid'
    else:
        linestyle = 'dotted'
    axs[0].axvline(seconds, color = 'g',linestyle = linestyle)
    axs[1].axvline(seconds, color = 'g',linestyle = linestyle)
    axs[2].axvline(seconds, color = 'g',linestyle = linestyle)
    axs[3].axvline(seconds, color = 'g',linestyle = linestyle)



axs[0].plot(gyro_time,gyro.iloc[:,1], color = 'r')
axs[0].grid(True)
axs[0].set_title("x")



axs[1].plot(gyro_time,gyro.iloc[:,2], color = 'r')
axs[1].grid(True)
axs[1].set_title("y")



axs[2].plot(gyro_time,gyro.iloc[:,3], color = 'r')
axs[2].grid(True)
axs[2].set_title("z")


axs[3].plot(gyro_length_valleys_time, np.array(gyro_length)[gyro_length_valleys], "o")
axs[3].plot(gyro_time,gyro_length, color = 'r')
axs[3].grid(True)
axs[3].set_title("length")




plt.grid(True)
plt.show()

In [16]:
fig, axs = plt.subplots(ncols = 1, nrows = 3,sharey=True,sharex=True)

fig.suptitle('gyro length with low points', fontsize=16)

fig.supxlabel("time in seconds")
fig.supylabel("gyro in °/s")

plt.subplots_adjust(hspace=0.3)

axs[0].plot(gyro_time,gyro_length, color = 'r')
axs[0].plot(gyro_length_valleys_time_all,np.array(gyro_length)[gyro_length_valleys_all], "x")

axs[0].set_title("unfiltered")



axs[1].plot(gyro_time,gyro_length, color = 'r')
axs[1].plot(gyro_length_valleys_time_filer,np.array(gyro_length)[gyro_length_valleys_filer], "x")
axs[1].set_title("prominence filter")



axs[2].plot(gyro_time,gyro_length, color = 'r')
axs[2].plot(gyro_length_valleys_time,np.array(gyro_length)[gyro_length_valleys], "x")

axs[2].set_title("prominence and height filter")


plt.xlim([10,30])
plt.savefig("filter_example.pdf",format="pdf")


plt.show()

In [79]:

fig, axs = plt.subplots(ncols = 1, nrows = 4,sharey=True,sharex=True)

fig.suptitle('accel ', fontsize=16)

fig.supxlabel("time in seconds")
fig.supylabel("accel in m/s^2")

for x, y in zip(beats.iloc[:,0], beats.iloc[:,1]):

    seconds = x#+ offset_seconds

    if (y == 1):
        linestyle = 'solid'
    else:
        linestyle = 'dotted'
    axs[0].axvline(seconds, color = 'g',linestyle = linestyle)
    axs[1].axvline(seconds, color = 'g',linestyle = linestyle)
    axs[2].axvline(seconds, color = 'g',linestyle = linestyle)
    axs[3].axvline(seconds, color = 'g',linestyle = linestyle)



axs[0].plot(accel_time,accel.iloc[:,1], color = 'r')
axs[0].grid(True)
axs[0].set_title("x")



axs[1].plot(accel_time,accel.iloc[:,2], color = 'r')
axs[1].grid(True)
axs[1].set_title("y")



axs[2].plot(accel_time,accel.iloc[:,3], color = 'r')
axs[2].grid(True)
axs[2].set_title("z")



axs[3].plot(accel_time,accel_length, color = 'r')
axs[3].grid(True)
axs[3].set_title("length")


plt.grid(True)
plt.show()

In [9]:
plt.close()
spf = wave.open(location + "/cut.wav")
spf.setpos(0)
signal = spf.readframes(-1)
signal = np.frombuffer(signal, "int16")
time = np.linspace(0,len(signal)/spf.getframerate()/2,len(signal))
plt.title("Waveform")
plt.xlabel("time in seconds")
plt.ylabel("amplitude")
plt.plot(time,signal/max(signal))

for x, y in zip(beats.iloc[:,0], beats.iloc[:,1]):

    seconds = x#* spf.getframerate() * 2#+ offset_seconds

    if (y == 1):
        linestyle = 'solid'
    else:
        linestyle = 'dotted'
    plt.axvline(seconds, color = 'g',linestyle = linestyle)
mpl.rcParams['agg.path.chunksize'] = 200
plt.xlim([8,20])
plt.savefig("waveform.pdf")
plt.show()


In [82]:
plt.close()